<a href="https://colab.research.google.com/github/Nazmul1932/SPL_3/blob/main/spl3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
!pip install pyLDAvis

from pprint import pprint
import time
import pandas as pd
import re
import string
import nltk
from gensim.models import CoherenceModel
import gensim
from gensim.utils import simple_preprocess as SIMPLE_PREPROCESS  
from nltk.stem import WordNetLemmatizer
from gensim.parsing.preprocessing import STOPWORDS as STOP_WORDS
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.porter import *
import numpy as np
from gensim import corpora,models
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
import pyLDAvis
import pyLDAvis.gensim_models # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

import collections

try:
    collectionsAbc = collections.abc
except AttributeError:
    collectionsAbc = collections 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [19]:
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 1000

In [20]:
csv_file = "/content/drive/MyDrive/eclipse_platform.csv"

bug_report = pd.read_csv(csv_file)
bug_report.head(1)

,Issue_id,Priority,Component,Duplicated_issue,Title,Description,Status,Resolution,Version,Created_time,Resolved_time
0,1,P3,Team,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...,CLOSED,FIXED,2,10/10/2001 21:34,2/9/2012 15:57


In [21]:
%%time

bug_report = bug_report.drop(columns=['Priority', 'Component', 'Status', 'Resolution', 'Version', 'Created_time', 'Resolved_time'])
bug_report = bug_report.dropna(axis=0, subset=['Description'])
bug_report[bug_report['Description'].map(len) > 4]

bug_report.head(1)

CPU times: user 78.3 ms, sys: 5.91 ms, total: 84.2 ms
Wall time: 183 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resource; \t- release project to CVS; \t- edit the *.gif resource with an external editor; e.g. PaintShop; \t- save and close external editor; \t- in Navigator open the icon resource and verify that your changes are there; \t- release project; \t\t-> nothing to release!; \t- in Navigator open the icon resource and verify that your changes are still there; ; \tProblem: because I never Refreshed from local; the workspace hasnt changed so Release didnt find anything.; \tHowever opening the resource with an external editor found the modified file on disk and showed the changes.; ; \tThe real problem occurs if Release actually finds something to release but you dont spot that some resources are missing.; \tThis is extremely error prone: one of my changes didnt made it into build 110 because of this!; ; NOTES:; EG (5/23/01 3:00:33 PM); \tRelease should do a refresh from local before doing the release.; \tMoving to VCM; ; ; KM (05/27/01 5:10:19 PM);...


In [23]:
%%time

bug_report["Description"] = bug_report["Description"].str.replace("Fixed in HEAD", "")
bug_report["Description"] = bug_report["Description"].str.replace("Has been marked as read-only", " ")

CPU times: user 380 ms, sys: 309 µs, total: 381 ms
Wall time: 381 ms


In [24]:
%%time

def cleanTextFromDescriptionAndTitle(report):

  report = report.replace('\t', '')
  report = re.sub('\w*\d\w*', '', report)
  report = report.lower()
  report = report.translate(str.maketrans('', '', string.punctuation))
    
  return report

bug_report['Description'] = bug_report['Description'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))
bug_report['Title'] = bug_report['Title'].apply(lambda x: cleanTextFromDescriptionAndTitle(x))

bug_report.loc[[0]]

CPU times: user 10.2 s, sys: 24.2 ms, total: 10.3 s
Wall time: 10.3 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,usability issue with external editors,setup a project that contains a gif resource release project to cvs edit the gif resource with an external editor eg paintshop save and close external editor in navigator open the icon resource and verify that your changes are there release project nothing to release in navigator open the icon resource and verify that your changes are still there problem because i never refreshed from local the workspace hasnt changed so release didnt find anything however opening the resource with an external editor found the modified file on disk and showed the changes the real problem occurs if release actually finds something to release but you dont spot that some resources are missing this is extremely error prone one of my changes didnt made it into build because of this notes eg pm release should do a refresh from local before doing the release moving to vcm km pm comments from jm in related email should not do this for free could have a setting which made it optoinal but...


In [25]:
%%time

def wordTokenizationAndLemmatization(report):

    new_report = []
    for token in SIMPLE_PREPROCESS(report):
        if token not in STOP_WORDS:
          if len(token) > 5:
            new_report.append(WordNetLemmatizer().lemmatize(token, pos='v'))
    return new_report

bug_report['Title'] = bug_report['Title'].apply(lambda x: wordTokenizationAndLemmatization(x))
bug_report['Description'] = bug_report['Description'].apply(lambda x: wordTokenizationAndLemmatization(x))

bug_report.head(1)

CPU times: user 20.8 s, sys: 193 ms, total: 21 s
Wall time: 21.1 s


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [26]:
%%time

duplicated_reports = bug_report.dropna(axis=0, subset=['Duplicated_issue'])
duplicated_reports.reset_index(drop=True, inplace=True)
duplicated_reports.head(1)

CPU times: user 4.28 ms, sys: 1.04 ms, total: 5.32 ms
Wall time: 7.34 ms


,Issue_id,Duplicated_issue,Title,Description
0,25,28126.0,"[default, connection]","[helpful, notion, default, repository, connection, connect, project, wouldnt, everytime, create, create, default, minimum, have, default, repository, helpful, pulldown, connection, wizard, default]"


In [27]:
%%time

master_reports = bug_report[bug_report.isnull().any(axis=1)]
master_reports.reset_index(drop=True, inplace=True)
master_reports.head(1)

CPU times: user 13.9 ms, sys: 0 ns, total: 13.9 ms
Wall time: 15.2 ms


,Issue_id,Duplicated_issue,Title,Description
0,1,NaN,"[usability, external, editors]","[project, contain, resource, release, project, resource, external, editor, paintshop, external, editor, navigator, resource, verify, change, release, project, release, navigator, resource, verify, change, problem, refresh, workspace, change, release, open, resource, external, editor, modify, show, change, problem, occur, release, actually, release, resources, miss, extremely, change, release, refresh, release, move, comment, relate, set, optoinal, mandatory, default, set, consider, workspaces, network, drive, effect, refresh, complete, workspace, runnable, runnable, release, optimal, builders, responsible, maintain, invariants, derive, resources, releasable, builders, release, invariants, maintain, release, inconsistent, summary, ensure, consistency, release, itpvcmwinnt, internal, compare, document, fail]"


In [28]:
%%time

dictionary = gensim.corpora.Dictionary(master_reports['Description'])
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

count = 0
for key, value in dictionary.iteritems():
  print(key, value)
  count += 1
  if count > 100:
      break


0 actually
1 builders
2 change
3 comment
4 compare
5 complete
6 consider
7 consistency
8 contain
9 default
10 derive
11 document
12 drive
13 editor
14 effect
15 ensure
16 external
17 extremely
18 fail
19 inconsistent
20 internal
21 maintain
22 mandatory
23 miss
24 modify
25 move
26 navigator
27 network
28 occur
29 open
30 optimal
31 problem
32 project
33 refresh
34 relate
35 release
36 resource
37 resources
38 responsible
39 runnable
40 set
41 show
42 summary
43 verify
44 workspace
45 workspaces
46 accomodate
47 agree
48 content
49 critical
50 dependent
51 discuss
52 display
53 doesnt
54 edit
55 editors
56 eventually
57 example
58 expect
59 general
60 graphic
61 map
62 need
63 outliner
64 possible
65 reference
66 remote
67 repositories
68 repository
69 result
70 sensible
71 specialize
72 vanilla
73 write
74 appear
75 arrows
76 attention
77 better
78 clarification
79 clearer
80 correctly
81 delete
82 deletion
83 distinction
84 especially
85 focus
86 indicate
87 indication
88 indicator
8

In [29]:
%%time

bag_of_words_corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in master_reports['Description']]
bag_of_words_corpus[0]

CPU times: user 1.73 s, sys: 39.2 ms, total: 1.77 s
Wall time: 1.77 s


[(0, 1),
 (1, 2),
 (2, 5),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 3),
 (14, 1),
 (15, 1),
 (16, 3),
 (17, 1),
 (18, 1),
 (19, 1),
 (20, 1),
 (21, 2),
 (22, 1),
 (23, 1),
 (24, 1),
 (25, 1),
 (26, 2),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 2),
 (32, 3),
 (33, 3),
 (34, 1),
 (35, 12),
 (36, 5),
 (37, 2),
 (38, 1),
 (39, 2),
 (40, 2),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 1),
 (4543, 2),
 (4544, 1),
 (4545, 1),
 (4546, 1),
 (4547, 1)]

In [30]:
%%time

bag_of_words_doc = bag_of_words_corpus[0]

for i in range(len(bag_of_words_doc)):
    print("{}. \"{}\" word seen {} time.".format(i+1, dictionary[bag_of_words_doc[i][0]], bag_of_words_doc[i][1]))
                                               

1. "actually" word seen 1 time.
2. "builders" word seen 2 time.
3. "change" word seen 5 time.
4. "comment" word seen 1 time.
5. "compare" word seen 1 time.
6. "complete" word seen 1 time.
7. "consider" word seen 1 time.
8. "consistency" word seen 1 time.
9. "contain" word seen 1 time.
10. "default" word seen 1 time.
11. "derive" word seen 1 time.
12. "document" word seen 1 time.
13. "drive" word seen 1 time.
14. "editor" word seen 3 time.
15. "effect" word seen 1 time.
16. "ensure" word seen 1 time.
17. "external" word seen 3 time.
18. "extremely" word seen 1 time.
19. "fail" word seen 1 time.
20. "inconsistent" word seen 1 time.
21. "internal" word seen 1 time.
22. "maintain" word seen 2 time.
23. "mandatory" word seen 1 time.
24. "miss" word seen 1 time.
25. "modify" word seen 1 time.
26. "move" word seen 1 time.
27. "navigator" word seen 2 time.
28. "network" word seen 1 time.
29. "occur" word seen 1 time.
30. "open" word seen 1 time.
31. "optimal" word seen 1 time.
32. "problem" wo

In [31]:
%%time

corpus = bag_of_words_corpus

lda_model = gensim.models.LdaMulticore(corpus, num_topics=10, id2word=dictionary, passes=20, workers=2, iterations=100)

pprint(lda_model.print_topics())

[(0,
  '0.109*"message" + 0.087*"bundle" + 0.055*"miss" + 0.041*"resolve" + '
  '0.035*"orgeclipseosgi" + 0.032*"plugin" + 0.030*"subentry" + '
  '0.029*"require" + 0.013*"unable" + 0.011*"fragment"'),
 (1,
  '0.047*"editor" + 0.027*"eclipse" + 0.026*"window" + 0.021*"reproduce" + '
  '0.017*"perspective" + 0.016*"button" + 0.016*"select" + 0.014*"windows" + '
  '0.012*"create" + 0.012*"reproducible"'),
 (2,
  '0.016*"return" + 0.013*"action" + 0.013*"change" + 0.013*"extension" + '
  '0.011*"create" + 0.009*"support" + 0.009*"provide" + 0.009*"plugin" + '
  '0.009*"element" + 0.008*"method"'),
 (3,
  '0.067*"eclipse" + 0.023*"version" + 0.022*"update" + 0.020*"feature" + '
  '0.010*"install" + 0.010*"problem" + 0.010*"instal" + 0.009*"reproduce" + '
  '0.009*"create" + 0.009*"plugin"'),
 (4,
  '0.083*"import" + 0.077*"public" + 0.051*"display" + 0.025*"static" + '
  '0.020*"string" + 0.020*"return" + 0.016*"snippet" + 0.014*"private" + '
  '0.013*"swtnone" + 0.013*"listener"'),
 (5,
 

In [32]:
%%time

print('\nPerplexity: ', lda_model.log_perplexity(bag_of_words_corpus))

coherence_model_lda = CoherenceModel(model=lda_model, texts=bug_report['Description'], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.343142729574823

Coherence Score:  0.6230628007248706
CPU times: user 27.3 s, sys: 288 ms, total: 27.6 s
Wall time: 27.2 s


In [33]:
%%time

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, bag_of_words_corpus, dictionary)
vis

CPU times: user 12 s, sys: 250 ms, total: 12.3 s
Wall time: 15.2 s


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.120781 -0.124783       1        1  16.543973
2      0.149350 -0.054354       2        1  15.516824
8      0.214680 -0.037100       3        1  15.421920
1      0.189901  0.010424       4        1  14.412333
7      0.185169  0.083870       5        1  10.331116
6     -0.138190 -0.161949       6        1   8.090949
4     -0.009283  0.299301       7        1   7.036309
9     -0.239507  0.089271       8        1   5.190286
0     -0.207396 -0.212861       9        1   4.093352
5     -0.265506  0.108181      10        1   3.362938, topic_info=                  Term          Freq         Total Category  logprob  loglift
372             source  14940.000000  14940.000000  Default  30.0000  30.0000
32             project  21646.000000  21646.000000  Default  29.0000  29.0000
116             method  19411.000000  19411.000000  Default  28.0000  28.0000
589            message  12693.000000  12693.000000  Default  27.0000  27.0000
761             import  10295.000000  10295.000000  Default  26.0000  26.0000
360             public   9577.000000   9577.000000  Default  25.0000  25.0000
103            eclipse  29164.000000  29164.000000  Default  24.0000  24.0000
13              editor  11979.000000  11979.000000  Default  23.0000  23.0000
484             bundle   6272.000000   6272.000000  Default  22.0000  22.0000
1341            thread   7709.000000   7709.000000  Default  21.0000  21.0000
52             display  10461.000000  10461.000000  Default  20.0000  20.0000
109             select  14500.000000  14500.000000  Default  19.0000  19.0000
1196            launch   5379.000000   5379.000000  Default  18.0000  18.0000
806          exception   6315.000000   6315.000000  Default  17.0000  17.0000
609             window   6735.000000   6735.000000  Default  16.0000  16.0000
358             plugin   8433.000000   8433.000000  Default  15.0000  15.0000
23                miss   3983.000000   3983.000000  Default  14.0000  14.0000
283             object   5362.000000   5362.000000  Default  13.0000  13.0000
4346       libjvmdylib   3330.000000   3330.000000  Default  12.0000  12.0000
101             dialog  11049.000000  11049.000000  Default  11.0000  11.0000
3241            daemon   3415.000000   3415.000000  Default  10.0000  10.0000
4365        javathread   3388.000000   3388.000000  Default   9.0000   9.0000
448             return   8006.000000   8006.000000  Default   8.0000   8.0000
361            resolve   2996.000000   2996.000000  Default   7.0000   7.0000
130            feature   6294.000000   6294.000000  Default   6.0000   6.0000
153            require   4013.000000   4013.000000  Default   5.0000   5.0000
248             update   9168.000000   9168.000000  Default   4.0000   4.0000
608             search   6044.000000   6044.000000  Default   3.0000   3.0000
4374     threadblocked   2856.000000   2856.000000  Default   2.0000   2.0000
780             static   3116.000000   3116.000000  Default   1.0000   1.0000
1174          download   2238.626324   2239.508123   Topic1  -4.8727   1.7988
3527       cwindowsdll   1455.294481   1456.175983   Topic1  -5.3034   1.7985
1247            script   1353.207267   1354.088681   Topic1  -5.3761   1.7985
1302            mirror    829.374562    830.256018   Topic1  -5.8656   1.7981
4243           equinox    741.862148    742.743655   Topic1  -5.9772   1.7980
792           password    599.104167    599.985531   Topic1  -6.1909   1.7977
1958           archive    554.614554    555.496072   Topic1  -6.2680   1.7976
1242            releng    536.232285    537.113716   Topic1  -6.3018   1.7975
2582          buildxml    501.158740    502.040133   Topic1  -6.3694   1.7974
27             network    446.623692    447.505149   Topic1  -6.4846   1.7972
2871          baseline    413.707386    414.588848   Topic1  -6.5612   1.7970
4540            kepler    390.913

In [42]:
unseen_document = "Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum."


bow_vector = topic=lda_model[dictionary.doc2bow(wordTokenizationAndLemmatization(unseen_document))]

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 10)))

Score: 0.5452271699905396	 Topic: 0.018*"control" + 0.017*"character" + 0.015*"change" + 0.014*"create" + 0.013*"widget" + 0.011*"problem" + 0.009*"selection" + 0.009*"string" + 0.008*"background" + 0.008*"example"
Score: 0.05054222792387009	 Topic: 0.016*"return" + 0.013*"action" + 0.013*"change" + 0.013*"extension" + 0.011*"create" + 0.009*"support" + 0.009*"provide" + 0.009*"plugin" + 0.009*"element" + 0.008*"method"
Score: 0.050539128482341766	 Topic: 0.075*"project" + 0.037*"select" + 0.029*"dialog" + 0.026*"change" + 0.025*"create" + 0.020*"workspace" + 0.019*"search" + 0.013*"resource" + 0.013*"folder" + 0.012*"delete"
Score: 0.05052955448627472	 Topic: 0.067*"eclipse" + 0.023*"version" + 0.022*"update" + 0.020*"feature" + 0.010*"install" + 0.010*"problem" + 0.010*"instal" + 0.009*"reproduce" + 0.009*"create" + 0.009*"plugin"
Score: 0.05052843317389488	 Topic: 0.047*"editor" + 0.027*"eclipse" + 0.026*"window" + 0.021*"reproduce" + 0.017*"perspective" + 0.016*"button" + 0.016*"se